# FastHAN练习与实践

fastHan是基于fastNLP与pytorch实现的中文自然语言处理工具，像spacy一样调用方便。

其内核为基于BERT的联合模型，其在13个语料库中进行训练，可处理中文分词、词性标注、依存句法分析、命名实体识别四项任务。

fastHan共有base与large两个版本，分别利用BERT的前四层与前八层。

base版本在总参数量150MB的情况下各项任务均有不错表现，large版本则接近甚至超越SOTA模型。

## 初始化

In [1]:
from fastHan import FastHan

用户首次初始化模型时模块将自动从服务器下载参数。模型默认初始化base版本，可修改model_type参数值为’large’加载large版本。

In [2]:
model = FastHan(model_type='base')

  0%|                                                                 | 213k/144M [00:00<01:16, 1.88MB/s]

http://212.129.155.247/fasthan/fasthan_base.zip not found in cache, downloading to C:\Users\bhe\AppData\Local\Temp\tmpaf6_hofg


100%|█████████████████████████████████████████████████████████████████| 144M/144M [00:29<00:00, 4.95MB/s]


Finish download from http://212.129.155.247/fasthan/fasthan_base.zip
Copy file to C:\Users\bhe\.fastNLP\fasthan\fasthan_base
loading vocabulary file C:\Users\bhe\.fastNLP\fasthan\fasthan_base\vocab.txt
Load pre-trained BERT parameters from file C:\Users\bhe\.fastNLP\fasthan\fasthan_base\model.bin.


## 输入句子

输入模型的可以是单独的字符串，也可是由字符串组成的列表。

如果输入的是列表，模型将一次性处理所有输入的字符串，所以请自行控制 batch size。

模型对句子进行依存分析、命名实体识别的简单例子如下

In [44]:
sentences = ['武磊再次在比赛中打进单刀球。']

In [45]:
print(model(sentences, 'CWS'))

[['武磊', '再次', '在', '比赛', '中', '打进', '单刀', '球', '。']]


In [46]:
answer = model(sentences, 'Parsing')

In [47]:
for i, sentence in enumerate(answer):
    print(i)
    for token in sentence:
        print('token: {0}, token_position: {1}, token_head: {2}, token_head_label: {3}'
        .format(token, token.pos, token.head, token.head_label))

0
token: 武磊, token_position: NR, token_head: 6, token_head_label: nsubj
token: 再次, token_position: AD, token_head: 6, token_head_label: advmod
token: 在, token_position: P, token_head: 6, token_head_label: prep
token: 比赛, token_position: NN, token_head: 5, token_head_label: lobj
token: 中, token_position: LC, token_head: 3, token_head_label: plmod
token: 打进, token_position: VV, token_head: 0, token_head_label: root
token: 单刀, token_position: NN, token_head: 8, token_head_label: nn
token: 球, token_position: NN, token_head: 6, token_head_label: dobj
token: 。, token_position: PU, token_head: 6, token_head_label: punct


其中，target参数可在'Parsing'、'CWS'、'POS'、'NER'四个选项中取值，模型将分别进行依存分析、分词、词性标注、命名实体识别任务,模型默认进行CWS任务。其中词性标注任务包含了分词的信息，而依存分析任务又包含了词性标注任务的信息。命名实体识别任务相较其他任务独立。

模型的输出是在fastHan模块中定义的sentence与token类。模型将输出一个由sentence组成的列表，而每个sentence又由token组成。每个token本身代表一个被分好的词，有pos、head、head_label、ner四项属性，代表了该词的词性、依存关系、命名实体识别信息。

如果分别运行CWS、POS、Parsing任务，模型输出的分词结果等可能存在冲突。如果想获得不冲突的各类信息，可以直接运行包含全部所需信息的那项任务。

模型的POS、Parsing任务均使用CTB标签集。NER使用msra标签集。

正如上图所示，模型基础的使用流程非常简单。此外，模型还可调整分词风格、切换设备。

### NER示例

In [25]:
sentences = ['特朗普再次在推特发表声明。', 
'B站官方发布声明，称此前已经与“巫师财经”签订长期内容合作协议，此次为后者单方面违约。对于这种行为，感到“十分震惊和遗憾”。']
ners = model(sentences, 'NER')
for i, ner in enumerate(ners):
    print(i)
    print(ner)

0
[['特朗普', 'NR']]
1
[['B站', 'NT']]


In [38]:
sentences = ['6月10日，《江门市进一步推进户籍制度改革实施方案（修订）》发布并将于7月5日正式实施，这标志着江门迈向“零门槛”落户。', 
'腾讯证券6月16日讯，美国三大股指周一低开高走，截至收盘全线大涨，道指抹去盘中762点跌幅涨超百点，因美联储表示，将开始广泛购买美国公司债券的组合。此前三大股指大幅低开，延续自上周以来的回调趋势。',
'纽约梅隆银行中国ADR指数收盘上涨0.71%。拼多多涨6.24%，京东涨2.62%，网易涨0.91%，百度涨0.46%，爱奇艺涨7.58%，哔哩哔哩涨6.52%，唯品会涨5.64%，跟谁学涨20.82%，微博涨2.96%，第九城市涨61.97%。']
ners = model(sentences, 'NER')
for i, ner in enumerate(ners):
    print(i)
    print(ner)

0
[['江门市', 'NS'], ['江门', 'NS']]
1
[['腾讯证券', 'NT'], ['美国', 'NS'], ['道指', 'NT'], ['美联储', 'NT'], ['美国', 'NS']]
2
[['纽约梅隆银行', 'NT'], ['中国', 'NS'], ['京东', 'NS'], ['网易', 'NT'], ['百度', 'NT'], ['爱奇艺', 'NT'], ['唯品会', 'NT']]


### 中文分词示例

In [36]:
sentences = ['特朗普再次在推特发表声明。', 
'B站官方发布声明，称此前已经与“巫师财经”签订长期内容合作协议，此次为后者单方面违约。对于这种行为，感到“十分震惊和遗憾”。']
cwses = model(sentences, 'CWS')
for i, cws in enumerate(cwses):
    print(i)
    print(cws)

0
['特朗普', '再次', '在', '推特', '发表', '声明', '。']
1
['B', '站', '官方', '发布', '声明', '，', '称', '此前', '已经', '与', '“', '巫师', '财经', '”', '签订', '长期', '内容', '合作', '协议', '，', '此', '次', '为', '后者', '单方面', '违约', '。', '对于', '这', '种', '行为', '，', '感到', '“', '十分', '震惊', '和', '遗憾', '”', '。']


### 词性标注示例

In [29]:
poses = model(sentences, 'POS')
for i, pos in enumerate(poses):
    print(i)
    print(pos)

0
[['特朗普', 'NR'], ['再次', 'AD'], ('在', 'P'), ['推特', 'NR'], ['发表', 'VV'], ['声明', 'NN'], ('。', 'PU')]
1
[['B站', 'NN'], ['官方', 'NN'], ['发布', 'VV'], ['声明', 'NN'], ('，', 'PU'), ('称', 'VV'), ['此前', 'NT'], ['已经', 'AD'], ('与', 'P'), ('“', 'PU'), ['巫师', 'NN'], ['财经', 'NN'], ('”', 'PU'), ['签订', 'VV'], ['长期', 'JJ'], ['内容', 'NN'], ['合作', 'NN'], ['协议', 'NN'], ('，', 'PU'), ('此', 'DT'), ('次', 'M'), ('为', 'P'), ['后者', 'NN'], ['单方面', 'AD'], ['违约', 'VV'], ('。', 'PU'), ['对于', 'P'], ('这', 'DT'), ('种', 'M'), ['行为', 'NN'], ('，', 'PU'), ['感到', 'VV'], ('“', 'PU'), ['十分', 'AD'], ['震惊', 'VV'], ('和', 'CC'), ['遗憾', 'VA'], ('”', 'PU'), ('。', 'PU')]


### 调整分词风格

模型是在13个语料库中进行训练，其中包含了10个分词语料库。

不同语料库的分词粒度均不同，如本模型默认的CTB语料库分词粒度较细。

如果想切换不同的粒度，可以使用模型的set_cws_style函数，例子如下：

In [35]:
sentences = ['武磊再次在比赛中打进单刀球。', '最后生还者第二章将在本周五发布']
model.set_cws_style('ctb')
cwses = model(sentences, 'CWS')
print(cwses)
model.set_cws_style('cnc')
print()
cwses = model(sentences, 'CWS')
print(cwses)
model.set_cws_style('ctb')

[['武磊', '再次', '在', '比赛', '中', '打进', '单刀', '球', '。'], ['最后', '生还者', '第二', '章', '将', '在', '本', '周五', '发布']]

[['武磊', '再次', '在', '比赛', '中', '打进', '单刀', '球', '。'], ['最后', '生还', '者', '第二', '章', '将', '在', '本', '周五', '发布']]


### 依存关系

In [40]:
sentences = ['武磊再次在比赛中打进单刀球。', '最后生还者第二章将在本周五发布', '苹果公司将在2020年下半财年发布支持5G的新款手机。']
answer=model(sentences,target="Parsing")
print(answer)
answer=model(sentences,target="NER")
print(answer)

[[['武磊', 6, 'nsubj', 'NR'], ['再次', 6, 'advmod', 'AD'], ['在', 6, 'prep', 'P'], ['比赛', 5, 'lobj', 'NN'], ['中', 3, 'plmod', 'LC'], ['打进', 0, 'root', 'VV'], ['单刀', 8, 'nn', 'NN'], ['球', 6, 'dobj', 'NN'], ['。', 6, 'punct', 'PU']], [['最后', 9, 'advmod', 'AD'], ['生还者', 4, 'dep', 'NN'], ['第二', 4, 'ordmod', 'OD'], ['章', 9, 'nsubj', 'M'], ['将', 9, 'advmod', 'AD'], ['在', 9, 'prep', 'P'], ['本', 8, 'det', 'DT'], ['周五', 6, 'pobj', 'NT'], ['发布', 0, 'root', 'VV']], [['苹果', 2, 'nn', 'NR'], ['公司', 8, 'nsubj', 'NN'], ['将', 8, 'advmod', 'AD'], ['在', 8, 'prep', 'P'], ['2020年', 7, 'nn', 'NT'], ['下半', 7, 'nn', 'NT'], ['财年', 4, 'pobj', 'NN'], ['发布', 0, 'root', 'VV'], ['支持', 14, 'rcmod', 'VV'], ['5', 11, 'nummod', 'CD'], ['G', 9, 'range', 'M'], ['的', 9, 'cpm', 'DEC'], ['新款', 14, 'amod', 'JJ'], ['手机', 8, 'dobj', 'NN'], ['。', 8, 'punct', 'PU']]]
[[['武磊', 'NR']], [], [['苹果公司', 'NT']]]


In [41]:
sentences = ['西门子将努力参与中国的三峡工程建设。']
answer=model(sentences,target="Parsing")
print(answer)
answer=model(sentences,target="NER")
print(answer)

[[['西门子', 4, 'nsubj', 'NR'], ['将', 4, 'advmod', 'AD'], ['努力', 4, 'advmod', 'AD'], ['参与', 0, 'root', 'VV'], ['中国', 9, 'assmod', 'NR'], ['的', 5, 'assm', 'DEG'], ['三峡', 9, 'nn', 'NR'], ['工程', 9, 'nn', 'NN'], ['建设', 4, 'dobj', 'NN'], ['。', 4, 'punct', 'PU']]]
[[['西门子', 'NT'], ['中国', 'NS'], ['三峡', 'NS']]]
